# What's Going on Here?

$$
\\
$$

## Web Scraping

- Download stock price data for the Euro Stoxx 50 constituents, as of Summer 2018.

$$
\\
$$

- Download Euro Stoxx 50 constituents ticker names from a table of Wikipedia 
https://en.wikipedia.org/wiki/EURO_STOXX_50

$$
\\
$$

- Work with python packages beautiful soup and requests.

# Learning Points: Data-Science Perspective

$$
\\
$$

- Beautiful Soup to extract content from HTML pages, here, ticker names of companies

$$
\\
$$

- Requests to extract market information for stocks, here end of day price quotes

# Learning Points: Finance Perspective

$$
\\
$$

- Get publicly available finance data for your companies of interest

$$
\\
$$

- There is probably more finance data available than you might have thought, :-)

$$
\\
$$

- Really 'hot' data such as option data with different strikes and maturities for firms of your interest are not publicly available :-(

# Python Work

## Downloading Euro Stoxx 50 Ticker List as of May 2018 from Wikipedia Table

In [1]:
#packages
import bs4 as bs #±beautifulsoup4
import requests

#extract Ticker Symbols from Wikipedia Table
#
def get_ES50Tickers_from_Wikipedia():
   #load article on Euro Stoxx 50 from Wikipedia
    website_url = requests.get('https://en.wikipedia.org/wiki/EURO_STOXX_50')
    soup = bs.BeautifulSoup(website_url.text) 

    #get all tables 'wikitable sortable'
    tables = soup.find_all('table', {'class':'wikitable sortable'}) 
    
    #Search through tables for the one with the headings 'Ticker' 
    
    for table in tables:
        ths = table.find_all('th')
        headings = [th.text.strip() for th in ths]
        #print(headings[0])     
        
        if headings[0] == 'Ticker': #name of each table's 1st column  
            break #now we are in the right table
    
    #get the tickers as a list
    tickers = [] #specify empty ticker list
    
    for row in table.findAll('tr')[1:]: #start with 2nd table row as 1st table row contains headings only (and no tickers)
        ticker = row.findAll('td')[0].text.strip() ##find 1st column in each row and strip out its text  
        tickers.append(ticker)  

    #save tickers into a txt file    
    with open("ES50tickers_2018_May.txt", "w") as f:
        print(tickers, file=f)

    return tickers     
    
 

**Test get_ES50Tickers_from_Wikipedia()**

In [2]:
tickers_ES50 = get_ES50Tickers_from_Wikipedia()

In [3]:
print(tickers_ES50)

['ADS.DE', 'AD.AS', 'AI.PA', 'AIR.PA', 'ALV.DE', 'ABI.BR', 'ASML.AS', 'AMA.MC', 'CS.PA', 'BBVA.MC', 'SAN.MC', 'BAS.DE', 'BAYN.DE', 'BMW.DE', 'BNP.PA', 'CRG.IR', 'SGO.PA', 'DAI.DE', 'DPW.DE', 'DTE.DE', 'ENEL.MI', 'ENGI.PA', 'ENI.MI', 'EOAN.DE', 'EL.PA', 'FRE.DE', 'BN.PA', 'IBE.MC', 'ITX.MC', 'INGA.AS', 'ISP.MI', 'LIN.DE', 'OR.PA', 'MC.PA', 'MUV2.DE', 'NOKIA.HE', 'ORA.PA', 'PHIA.AS', 'SAF.PA', 'SAN.PA', 'SAP.DE', 'SU.PA', 'SIE.DE', 'GLE.PA', 'TEF.MC', 'FP.PA', 'URW.AS', 'UNA.AS', 'DG.PA', 'VIV.PA', 'VOW.DE']


## Download All Daily Stock Price Data From Yahoo-Finance for Ticker List  

$$
\\
$$



In [4]:
#packages
import datetime as dt
import pandas as pd
import pandas_datareader as web

#get all stock price data that yahoo-finance offers
def get_YahooData(tickers_ES50):
    ##input: list of ticker symbols
    ##output: yahoo-finance data for tickers, daily frequency, returns as pandas data frame; also: csv is created
    
    #set start and end time
    t0 = dt.datetime(2000,1,1)
    T  = dt.datetime(2020, 12, 31)

    #get all stock price data that yahoo-finance offers 
    f_yh = web.DataReader(tickers_ES50, 'yahoo', t0, T)
    ##convert to pandas data frame
    f_yh = pd.DataFrame(f_yh)

    ##save to csv
    f_yh.to_csv('ES50_yahooData_Dec2020.csv')

    return f_yh

 



/Users/maximulrich/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


**Test get_YahooData(tickers_ES50)**

In [5]:
YahooData_d  = get_YahooData(tickers_ES50)

/Users/maximulrich/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/base.py:270: SymbolWarning: Failed to read symbol: 'AMA.MC', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


In [6]:
print(YahooData_d)

Attributes   Adj Close                                                \
Symbols         ADS.DE      AD.AS       AI.PA     AIR.PA      ALV.DE   
Date                                                                   
2000-01-03    9.737806        NaN   10.226519        NaN  145.655563   
2000-01-04    9.394521        NaN    9.772677        NaN  145.655563   
2000-01-05    9.171777        NaN    9.681908        NaN  144.967438   
2000-01-06    9.564853        NaN   10.377778        NaN  150.931305   
2000-01-07    9.433829        NaN   10.250728        NaN  155.128922   
...                ...        ...         ...        ...         ...   
2020-11-25  273.600006  23.389999  135.899994  91.500000  199.199997   
2020-11-26  274.700012  23.580000  137.600006  89.800003  199.059998   
2020-11-27  271.200012  23.809999  138.100006  90.000000  198.139999   
2020-11-30  267.299988  23.990000  137.649994  87.730003  197.660004   
2020-12-01  271.500000  23.879999  137.050003  88.620003  199.53

**Quick Look into YahooData_d PandasDataFrame**

In [7]:
YahooData_d.describe()

Attributes    Adj Close                                                      \
Symbols          ADS.DE        AD.AS        AI.PA       AIR.PA       ALV.DE   
count       5347.000000  3094.000000  5372.000000  4915.000000  5337.000000   
mean          70.481704    13.700283    51.788938    36.586018    95.474938   
std           74.075997     5.340404    33.864013    31.390323    51.349644   
min            6.420244     6.355588     7.866548     4.512766    21.141090   
25%           21.237410     8.413906    19.021725    13.534597    50.757236   
50%           36.682041    12.569590    44.338951    21.147314    79.063835   
75%           76.306358    17.898855    77.877705    51.778330   141.142502   
max          309.818024    26.330000   142.750000   134.595078   218.202454   

Attributes                                                                   \
Symbols          ABI.BR      ASML.AS        CS.PA      BBVA.MC       SAN.MC   
count       5125.000000  5382.000000  5374.000000  5361.000000  5377.000000   
mean          40.198182    61.939650     9.977829     5.464653     3.405262   
std           29.193960    71.837078     5.431234     1.419285     0.931003   
min            5.481520     3.916246     1.694767     2.160000     1.297446   
25%           12.840903    15.134487     5.797308     4.494747     2.665768   
50%           29.266012    29.759068     8.190170     5.341985     3.368664   
75%           66.174873    81.983505    14.475024     6.329268     4.208636   
max          101.662155   372.799988    22.369606     9.559817     5.331194   

Attributes  ...        Volume                                            \
Symbols     ...        SIE.DE        GLE.PA        TEF.MC         FP.PA   
count       ...  5.347000e+03  2.160000e+03  5.361000e+03  5.364000e+03   
mean        ...  4.089459e+06  4.910815e+06  3.858995e+07  8.267252e+06   
std         ...  2.895701e+06  2.428412e+06  3.420402e+07  5.546999e+06   
min         ...  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%         ...  2.134320e+06  3.398728e+06  2.016506e+07  5.175618e+06   
50%         ...  3.361476e+06  4.342170e+06  3.046309e+07  7.101443e+06   
75%         ...  5.231077e+06  5.784609e+06  4.477269e+07  1.001662e+07   
max         ...  4.145761e+07  2.705006e+07  5.276800e+08  1.507851e+08   

Attributes                                                          \
Symbols           URW.AS        UNA.AS         DG.PA        VIV.PA   
count       5.344000e+03  5.339000e+03  5.371000e+03  5.372000e+03   
mean        3.569225e+05  5.863820e+06  1.895287e+06  5.437345e+06   
std         4.224762e+05  6.155483e+06  1.259983e+06  3.433761e+06   
min         0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%         1.355292e+05  1.324500e+04  1.159817e+06  3.369223e+06   
50%         2.799510e+05  5.334552e+06  1.638422e+06  4.773811e+06   
75%         4.167505e+05  8.813800e+06  2.286197e+06  6.546073e+06   
max         7.081690e+06  6.291250e+07  1.808752e+07  5.674013e+07   

Attributes                       
Symbols           VOW.DE AMA.MC  
count       5.347000e+03    0.0  
mean        1.126243e+06    NaN  
std         1.705235e+06    NaN  
min         0.000000e+00    NaN  
25%         8.900100e+04    NaN  
50%         2.783780e+05    NaN  
75%         1.772112e+06    NaN  
max         1.859053e+07    NaN  

[8 rows x 306 columns]

## Clean YahooData_d to Meet Our Specific Needs

$$
\\
$$

- 1. Keep 'Adj Close Prices' only (i.e. delete Volume information, although that is interesting in its own right)

$$
\\
$$

- 2. Eliminate Tickers with more than 100 NaNs (out of roughly 5000 trading days). Reason: these are tickers that started trading late and for the upcoming analysis we want a long time-series sample

$$
\\
$$

- 3. Linearly interpolate NaNs that occur in the middle of time series. Reason: as trading took place we anticipate these NaNs correspond to public holidays or recording errors

 
 

In [8]:
#Cleaning
#
def Get_AdjPrice_cleaned(YahooData_d):
    ##create one data frame with Adj Close prices only  
    P_ES50_d = YahooData_d['Adj Close']
   
    #ignore tickers with 100 or more NaN
    P_ES50_d_s = P_ES50_d.dropna(axis=1, thresh=(P_ES50_d.shape[0]-100))
    
    #linear interpolation of neighboring values
    P_ES50_d_s = P_ES50_d_s.interpolate(method='linear', limit_direction='forward')
    
    
    return P_ES50_d_s


 

**Test Function Get_AdjPrice_cleaned(YahooData_d)**

In [9]:
P_ES50_d_cleaned = Get_AdjPrice_cleaned(YahooData_d)

**What's inside P_ES50_d_cleaned:**

In [10]:
print(P_ES50_d_cleaned)

Symbols         ADS.DE       AI.PA      ALV.DE     ASML.AS      CS.PA  \
Date                                                                    
2000-01-03    9.737806   10.226519  145.655563   28.812193   5.275198   
2000-01-04    9.394521    9.772677  145.655563   27.428837   5.081253   
2000-01-05    9.171777    9.681908  144.967438   24.812944   5.003675   
2000-01-06    9.564853   10.377778  150.931305   23.077412   4.964885   
2000-01-07    9.433829   10.250728  155.128922   24.800396   5.120043   
...                ...         ...         ...         ...        ...   
2020-11-25  273.600006  135.899994  199.199997  355.000000  19.870001   
2020-11-26  274.700012  137.600006  199.059998  357.450012  19.788000   
2020-11-27  271.200012  138.100006  198.139999  366.350006  19.732000   
2020-11-30  267.299988  137.649994  197.660004  363.850006  19.712000   
2020-12-01  271.500000  137.050003  199.539993  372.799988  19.716000   

Symbols      BBVA.MC    SAN.MC     BAS.DE    BAYN.

**More Condensed View into P_ES50_d_cleaned**

In [11]:
P_ES50_d_cleaned.head()

Symbols,ADS.DE,AI.PA,ALV.DE,ASML.AS,CS.PA,BBVA.MC,SAN.MC,BAS.DE,BAYN.DE,BMW.DE,...,SAP.DE,SU.PA,SIE.DE,TEF.MC,FP.PA,URW.AS,UNA.AS,DG.PA,VIV.PA,VOW.DE
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,9.737806,10.226519,145.655563,28.812193,5.275198,5.311121,2.726521,8.069616,25.103666,12.766329,...,27.296059,14.996898,65.977577,9.433228,7.150976,11.232464,6.060124,2.540235,32.128925,35.848274
2000-01-04,9.394521,9.772677,145.655563,27.428837,5.081253,5.172528,2.661833,7.841083,24.090979,12.251177,...,26.000206,14.465379,63.312019,9.044588,6.890940,11.421152,5.958649,2.392858,30.581442,34.871235
2000-01-05,9.171777,9.681908,144.967438,24.812944,5.003675,5.056392,2.597143,7.541625,23.451403,12.008748,...,22.862511,13.857899,61.169628,8.621984,6.717582,11.410366,6.088297,2.447227,29.844526,33.572803
2000-01-06,9.564853,10.377778,150.931305,23.077412,4.964885,5.088242,2.597143,7.569995,23.531343,11.969790,...,23.604174,14.560286,60.421505,8.629522,6.522557,11.752708,6.312648,2.528802,29.494482,33.424961
2000-01-07,9.433829,10.250728,155.128922,24.800396,5.120043,5.120093,2.712145,7.880485,24.512018,11.948142,...,28.619501,14.826064,62.852222,8.637059,6.555059,11.679924,6.403975,2.610379,29.733999,34.389149


In [12]:
P_ES50_d_cleaned.shape 

(5398, 42)

In [13]:
P_ES50_d_cleaned.describe()

Symbols,ADS.DE,AI.PA,ALV.DE,ASML.AS,CS.PA,BBVA.MC,SAN.MC,BAS.DE,BAYN.DE,BMW.DE,...,SAP.DE,SU.PA,SIE.DE,TEF.MC,FP.PA,URW.AS,UNA.AS,DG.PA,VIV.PA,VOW.DE
count,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,...,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000
mean,70.747515,51.795325,95.552113,62.078475,9.981648,5.468254,3.407595,33.001511,46.520138,39.642782,...,47.166147,36.352238,62.869558,6.678828,24.658006,76.432586,21.049790,33.711720,15.065091,97.675456
std,74.265521,33.876409,51.382389,71.997333,5.428241,1.420201,0.931258,21.713333,28.298899,22.726147,...,29.150049,23.642531,26.177960,1.687240,10.649266,47.434319,14.134203,25.738976,7.351051,56.910021
min,6.420244,7.866548,21.141090,3.916246,1.694767,2.160000,1.297446,5.241868,5.958626,10.099645,...,7.414098,7.826219,17.881168,2.794000,6.473800,11.232464,4.621459,2.104624,3.625609,19.545403
25%,21.358879,19.049447,50.782305,15.157712,5.797308,4.498434,2.666304,11.557059,23.377963,18.499079,...,26.640374,14.632362,39.388595,5.414121,17.181536,34.557536,8.774760,13.560169,10.107083,36.147947
50%,36.804817,44.338951,79.080940,29.915335,8.196865,5.346579,3.371315,27.583562,37.640242,28.927094,...,32.992516,29.992846,58.479845,6.833950,24.255630,68.821239,15.416957,27.144482,12.690515,105.605309
75%,76.569727,77.914181,141.177006,82.021349,14.486954,6.330806,4.211433,53.029675,68.943710,62.688653,...,61.232727,53.580630,81.717751,7.828719,32.687067,114.393669,32.615293,48.979534,19.194988,142.367569
max,309.818024,142.750000,218.202454,372.799988,22.369606,9.559817,5.331194,78.570663,119.399010,94.470192,...,142.259995,120.550003,120.099998,12.300961,49.498737,170.479172,54.818455,103.453598,52.172565,407.243500


In [14]:
P_ES50_d_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5398 entries, 2000-01-03 to 2020-12-01
Data columns (total 42 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ADS.DE    5398 non-null   float64
 1   AI.PA     5398 non-null   float64
 2   ALV.DE    5398 non-null   float64
 3   ASML.AS   5398 non-null   float64
 4   CS.PA     5398 non-null   float64
 5   BBVA.MC   5398 non-null   float64
 6   SAN.MC    5398 non-null   float64
 7   BAS.DE    5398 non-null   float64
 8   BAYN.DE   5398 non-null   float64
 9   BMW.DE    5398 non-null   float64
 10  BNP.PA    5398 non-null   float64
 11  CRG.IR    5397 non-null   float64
 12  SGO.PA    5398 non-null   float64
 13  DAI.DE    5398 non-null   float64
 14  DTE.DE    5398 non-null   float64
 15  ENEL.MI   5398 non-null   float64
 16  ENGI.PA   5398 non-null   float64
 17  ENI.MI    5398 non-null   float64
 18  EOAN.DE   5398 non-null   float64
 19  FRE.DE    5398 non-null   float64
 20  BN.PA     53

**Store P_ES50_d_cleaned in a csv File**

In [15]:
#Write AdjPrice Data into csv
P_ES50_d_cleaned.to_csv('P_ES50_d_cleaned_Dec_2020.csv')